In [2]:
import torch
import torchvision

import sys
import os
import argparse
import warnings
import numpy as np
#!pip install pandas
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout


from sklearn.neighbors import KernelDensity
from keras.models import load_model

2024-12-29 15:46:40.582259: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print(torch.__version__)
file_path = "../data/nsl/"

2.2.2


### Data Preprocessing

In [31]:
df = pd.read_csv(file_path+"KDDTest+.txt", header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


In [20]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])


In [32]:

df.columns = columns
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


In [33]:
def get_unique(df, column_name = 'attack'):
    unique_vals = df[column_name].unique()
    value_counts = df[column_name].value_counts()
    print(f"Unique Values ({len(unique_vals)}): {unique_vals}\n")
    print(f"Value Counts:\n{value_counts}\n{'='*40}\n")
get_unique(df)

Unique Values (38): ['neptune' 'normal' 'saint' 'mscan' 'guess_passwd' 'smurf' 'apache2'
 'satan' 'buffer_overflow' 'back' 'warezmaster' 'snmpgetattack'
 'processtable' 'pod' 'httptunnel' 'nmap' 'ps' 'snmpguess' 'ipsweep'
 'mailbomb' 'portsweep' 'multihop' 'named' 'sendmail' 'loadmodule' 'xterm'
 'worm' 'teardrop' 'rootkit' 'xlock' 'perl' 'land' 'xsnoop' 'sqlattack'
 'ftp_write' 'imap' 'udpstorm' 'phf']

Value Counts:
attack
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail           

In [34]:
attack_n = []
for i in df.attack :
  if i == 'normal':
    attack_n.append("normal")
  else:
    attack_n.append("attack")
df['attack'] = attack_n

get_unique(df)

Unique Values (2): ['attack' 'normal']

Value Counts:
attack
attack    12833
normal     9711
Name: count, dtype: int64



In [35]:
cat_features = df.select_dtypes(include='object').columns
cat_features

Index(['protocol_type', 'service', 'flag', 'attack'], dtype='object')

In [5]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
#clm=['protocol_type', 'service', 'flag', 'attack']
for x in cat_features:
    print(x)
    df[x]=le.fit_transform(df[x])

NameError: name 'cat_features' is not defined

In [42]:
# Save the DataFrame to a CSV file
df.to_csv('data/nsl/test.csv', index=False)

In [43]:
df.shape

(22544, 43)

In [2]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('data/nsl/test.csv')
y = df["attack"]
X = df.drop(["attack"], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, y,test_size=0.1,random_state=43) 

FileNotFoundError: [Errno 2] No such file or directory: 'data/nsl/test.csv'

In [7]:
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
df = pd.read_csv('data/nsl/test.csv')
y = df["attack"]
X = df.drop(["attack"], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, y,test_size=0.1,random_state=43) 
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) # we use only transform in order to prevent data leakage

FileNotFoundError: [Errno 2] No such file or directory: 'data/nsl/test.csv'

In [47]:
X_train[0].shape

(42,)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(64, input_dim=42, activation='relu'),
    Dense(128, activation='relu'),
    # Dropout(0.5),  # Uncomment for dropout
    Dense(64, activation='relu'),
    # Dropout(0.5),  # Uncomment for dropout
    Dense(1, activation='sigmoid')  # For binary classification
])

2024-12-15 19:07:58.789467: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/fuad/miniforge3/envs/nids/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [50]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9489 - loss: 0.1538 - val_accuracy: 0.9911 - val_loss: 0.0316
Epoch 2/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9916 - loss: 0.0251 - val_accuracy: 0.9947 - val_loss: 0.0215
Epoch 3/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9942 - loss: 0.0204 - val_accuracy: 0.9947 - val_loss: 0.0178
Epoch 4/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9953 - loss: 0.0144 - val_accuracy: 0.9956 - val_loss: 0.0159
Epoch 5/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9937 - loss: 0.0158 - val_accuracy: 0.9951 - val_loss: 0.0162
Epoch 6/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9957 - loss: 0.0131 - val_accuracy: 0.9951 - val_loss: 0.0179
Epoch 7/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9956 - loss: 0.0128 - val_accuracy: 0.9956 - val_loss: 0.0175
Epoch 8/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9948 - loss: 0.0156 - val_accuracy: 0.

In [ ]:
print("Y_test shape: ", y_test.shape)

Y_test shape:  (2255,)


### Detect AE

In [4]:
import os
import argparse
import warnings
import numpy as np
from sklearn.neighbors import KernelDensity
from keras.models import load_model
import sys
sys.path.append('../')  # Add the parent directory to the Python path
from util import (get_data, get_noisy_samples, get_mc_predictions,
                         get_deep_representations, score_samples, normalize,
                         train_lr, compute_roc)

In [5]:

# Optimal KDE bandwidths that were determined from CV tuning
BANDWIDTHS = {'mnist': 1.20, 'cifar': 0.26, 'svhn': 1.00, 'nsl': 1.00}
class ArgsClass:
    def __init__(self):
        self.dataset = 'nsl'
        self.attack = 'fgsm'
        #self.dataset = 'nsl'
        self.batch_size = 256
        #self.dataset = 'nsl'

    def my_method(self):
        print(self.dataset)

args = ArgsClass()
args.dataset

'nsl'

In [21]:
from manda.util import *
print('Loading the data and model...')
# Load the model
model = load_model('../data/model_%s.h5' % args.dataset)
# Load the dataset
X_train, Y_train, X_test, Y_test = get_data(args.dataset)
print("Original X_test shape:", X_test.shape)
X_test_adv = np.load('../data/Adv_%s_%s.npy' % (args.dataset,
                                                        args.attack))
# Craft an equal number of noisy samples
X_test_noisy = get_noisy_samples(X_test, X_test_adv, args.dataset,
                                    args.attack)
print(type(X_test_adv))
print("Original X_test_adv shape:", X_test_adv.shape)

Loading the data and model...
Original X_test shape: (22544, 42)
keep features..........4 ....nsl
<class 'numpy.ndarray'>
Original X_test_adv shape: (22544, 42)


/Users/fuad/Documents/GitHubRepos/artifacts/manda/../manda/util.py:259: UserWarning: Using pre-set Gaussian scale sizes to craft noisy samples. If you've altered the eps/eps-iter parameters of the attacks used, you'll need to update these. In the future, scale sizes will be inferred automatically from the adversarial samples.
  warnings.warn("Using pre-set Gaussian scale sizes to craft noisy "


In [22]:
# Check model accuracies on each sample type
for s_type, dataset in zip(['normal', 'noisy', 'adversarial'],
                            [X_test, X_test_noisy, X_test_adv]):
    _, acc = model.evaluate(dataset, Y_test, batch_size=args.batch_size,
                            verbose=0)
    print("Model accuracy on the %s test set: %0.2f%%" %
            (s_type, 100 * acc))
    # Compute and display average perturbation sizes
    if not s_type == 'normal':
        l2_diff = np.linalg.norm(
            dataset.reshape((len(X_test), -1)) -
            X_test.reshape((len(X_test), -1)),
            axis=1
        ).mean()
        print("Average L-2 perturbation size of the %s test set: %0.2f" %
                (s_type, l2_diff))

Model accuracy on the normal test set: 84.48%
Model accuracy on the noisy test set: 77.04%
Average L-2 perturbation size of the noisy test set: 4.69
Model accuracy on the adversarial test set: 15.56%
Average L-2 perturbation size of the adversarial test set: 5.18


In [ ]:
import numpy as np
predictions = model.predict(X_test)
preds_test = np.argmax(predictions, axis=1)

inds_correct = np.where(preds_test == Y_test.argmax(axis=1))[0]

indexs = random_select(len(X_test), len(inds_correct))
clean_data = X_test[indexs]
print("clean data shape: ", clean_data.shape)

X_test = X_test[inds_correct]
X_test_noisy = X_test_noisy[inds_correct]
X_test_adv = X_test_adv[inds_correct]
print("preds_test shape: ", preds_test.shape)
print("updated X_test_adv shape:", X_test_adv.shape)

705/705 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
preds_test shape:  (22544,)
updated X_test_adv shape: (19045, 42)


In [24]:
## Get Bayesian uncertainty scores
print('Getting Monte Carlo dropout variance predictions...')
uncerts_normal = get_mc_predictions(model, X_test, batch_size=args.batch_size).var(axis=0).mean(axis=1)
uncerts_noisy = get_mc_predictions(model, X_test_noisy, batch_size=args.batch_size).var(axis=0).mean(axis=1)
uncerts_adv = get_mc_predictions(model, X_test_adv, batch_size=args.batch_size).var(axis=0).mean(axis=1)


Getting Monte Carlo dropout variance predictions...


100%|██████████| 50/50 [00:18<00:00,  2.72it/s]


In [63]:
uncerts_clean = get_mc_predictions(model, clean_data, batch_size=args.batch_size).var(axis=0).mean(axis=1)
X_test_clean_features = get_deep_representations(model, clean_data, batch_size=args.batch_size)


100%|██████████| 50/50 [00:19<00:00,  2.54it/s]

1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━

In [25]:
## Get KDE scores
# Get deep feature representations
print('Getting deep feature representations...')
X_train_features = get_deep_representations(model, X_train, batch_size=args.batch_size)
X_test_normal_features = get_deep_representations(model, X_test, batch_size=args.batch_size)
X_test_noisy_features = get_deep_representations(model, X_test_noisy, batch_size=args.batch_size)
X_test_adv_features = get_deep_representations(model, X_test_adv, batch_size=args.batch_size)


Getting deep feature representations...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━

In [29]:
numClass = len(np.unique(Y_train))
print("--------------number of class label: ", numClass)
# Train one KDE per class
print('Training KDEs...')
class_inds = {}
for i in range(Y_train.shape[1]):
    class_inds[i] = np.where(Y_train.argmax(axis=1) == i)[0]

class_inds

--------------number of class label:  2
Training KDEs...


In [32]:
kdes = {}

for i in range(Y_train.shape[1]):
    kdes[i] = KernelDensity(kernel='gaussian',
                            bandwidth=BANDWIDTHS[args.dataset]) \
        .fit(X_train_features[class_inds[i]])

In [ ]:
preds_test_normal = np.argmax(model.predict(X_test), axis=1)
preds_test_noisy = np.argmax(model.predict(X_test_noisy), axis=1)
preds_test_adv = np.argmax(model.predict(X_test_adv), axis=1)

596/596 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
596/596 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
596/596 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [64]:

preds_test_clean = np.argmax(model.predict(clean_data), axis=1)

densities_clean = score_samples(
    kdes,
    X_test_clean_features,
    preds_test_clean
)


596/596 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2024-12-29 17:21:19.023393: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 17:21:19.023400: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 17:21:19.023393: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 17:21:19.023393: I tensorflow/core/platform/cpu_featu

In [34]:
# Get density estimates
print('computing densities...')
densities_normal = score_samples(
    kdes,
    X_test_normal_features,
    preds_test_normal
)
densities_noisy = score_samples(
    kdes,
    X_test_noisy_features,
    preds_test_noisy
)
densities_adv = score_samples(
    kdes,
    X_test_adv_features,
    preds_test_adv
)

computing densities...


2024-12-29 16:05:36.102689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 16:05:36.102688: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 16:05:36.102715: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 16:05:36.102728: I tensorflow/core/platform/cpu_featu

In [68]:
def normalize_std(normal, adv, noisy, clean):
    """
    TODO
    :param normal:
    :param adv:
    :param noisy:
    :return:
    """
    n_samples = len(normal)
    scaler = StandardScaler()
    #total = scaler.fit_transform(np.concatenate((normal, adv, noisy, clean)))
    total = scale(np.concatenate((normal, adv, noisy, clean)))

    return total[:n_samples], total[n_samples:2*n_samples], total[2*n_samples:3*n_samples], total[3*n_samples:]


In [70]:
## Z-score the uncertainty and density values
uncerts_normal_z, uncerts_adv_z, uncerts_noisy_z, uncerts_clean_z = normalize_std(
    uncerts_normal,
    uncerts_adv,
    uncerts_noisy,
    uncerts_clean
)
densities_normal_z, densities_adv_z, densities_noisy_z, densities_clean_z = normalize_std(
    densities_normal,
    densities_adv,
    densities_noisy,
    densities_clean
)

In [37]:
## Build detector
values, labels, lr = train_lr(
    densities_pos=densities_adv_z,
    densities_neg=np.concatenate((densities_normal_z, densities_noisy_z)),
    uncerts_pos=uncerts_adv_z,
    uncerts_neg=np.concatenate((uncerts_normal_z, uncerts_noisy_z))
)

In [71]:
def get_neg_values_for_lr(densities_neg, uncerts_neg):
    """
    :param densities_neg:
    :param uncerts_neg:
    :return:
    """
    values_neg = np.concatenate(
        (densities_neg.reshape((1, -1)),
         uncerts_neg.reshape((1, -1))),
        axis=0).transpose([1, 0])

    #values = np.concatenate((values_neg, values_pos))
    labels = np.zeros_like(densities_neg)

    return values_neg, labels

vals, labs = get_neg_values_for_lr(densities_clean_z, uncerts_clean_z)

In [ ]:
accuracy = lr.score(vals, labs)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9994749278025729


In [75]:
acc = lr.score(vals, labs)
print("Model accuracy on the purified (remove detect adv) test set: %0.2f%%" % (100 * acc))


Model accuracy on the purified (remove detect adv) test set: 99.95%


In [74]:
from sklearn import metrics

y_pred = lr.predict(vals)

# Accuracy
accuracy = metrics.accuracy_score(labs, y_pred)
print(f"Accuracy: {accuracy}")

# Precision
precision = metrics.precision_score(labs, y_pred)
print(f"Precision: {precision}")

# Recall
recall = metrics.recall_score(labs, y_pred)
print(f"Recall: {recall}")

# F1-score
f1 = metrics.f1_score(labs, y_pred)
print(f"F1-score: {f1}")

# Confusion matrix
cm = metrics.confusion_matrix(labs, y_pred)
print(f"Confusion Matrix:\n{cm}")

# Classification report (provides precision, recall, F1-score, and support for each class)
cr = metrics.classification_report(labs, y_pred)
print(f"Classification Report:\n{cr}")

#ROC AUC Score
#roc_auc = metrics.roc_auc_score(labs, lr.predict_proba(vals)[:, 1])
#print(f"ROC AUC Score:\n{roc_auc}")

Accuracy: 0.9994749278025729
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Confusion Matrix:
[[19035    10]
 [    0     0]]
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     19045
         1.0       0.00      0.00      0.00         0

    accuracy                           1.00     19045
   macro avg       0.50      0.50      0.50     19045
weighted avg       1.00      1.00      1.00     19045



/Users/fuad/miniforge3/envs/nids/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fuad/miniforge3/envs/nids/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fuad/miniforge3/envs/nids/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fuad/minifo

In [38]:
## Evaluate detector
# Compute logistic regression model predictions
probs = lr.predict_proba(values)[:, 1]
# Compute AUC
n_samples = len(X_test)
# The first 2/3 of 'probs' is the negative class (normal and noisy samples),
# and the last 1/3 is the positive class (adversarial samples).
_, _, auc_score = compute_roc(
    probs_neg=probs[:2 * n_samples],
    probs_pos=probs[2 * n_samples:]
)
print('Detector ROC-AUC score: %0.4f' % auc_score)


Detector ROC-AUC score: 0.9682


In [78]:
fpr, tpr, _ = roc_curve(labels, probs)
print(fpr)
tpr

[0.00000000e+00 0.00000000e+00 2.62536099e-05 ... 4.46075085e-01
 4.46127593e-01 1.00000000e+00]


array([0.00000000e+00, 5.25072197e-05, 5.25072197e-05, ...,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00])

In [ ]:
y_pred = lr.predict(values)
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [81]:
# Accuracy
accuracy = metrics.accuracy_score(labels, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6666666666666666


In [84]:
print('Detector Model Accuracy:: %0.2f' % (100*accuracy))

Detector Model Accuracy:: 66.67


In [33]:
i=8
i & 1

0

In [34]:
i >> 1

4